In [1]:
%%capture
! pip install pyLDAvis
! pip install pymorphy2

In [2]:
from gensim.test.utils import common_corpus
from gensim.models import LdaSeqModel
import gensim
import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import warnings
import json
import pymorphy2
import nltk
import re 
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
import nltk
from nltk import ngrams
import time
import dateutil.parser as dp
nltk.download('stopwords')
import pandas as pd
from sklearn.model_selection import train_test_split
import datetime

russian_stopwords = stopwords.words("russian")
warnings.filterwarnings("ignore", category=DeprecationWarning)

/opt/conda/lib/python3.9/site-packages/past/types/oldstr.py:23: DeprecationWarning: invalid escape sequence \d
  """
/opt/conda/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
[nltk_data] Error loading stopwords: <urlopen error [Errno 104]
[nltk_data]     Connection reset by peer>


In [4]:
row_data = pd.read_json("/home/jovyan/work/outputs.jsonl", lines = True, orient = 'records')

In [ ]:
row_data

In [6]:
class Preparator():
    def __init__(self):
        self.morph = pymorphy2.MorphAnalyzer()
        self.tokenizer = nltk.WordPunctTokenizer()
        self.regexp = re.compile('[-%-,_\.\n\\\t\r/\W]')  # оставляет только буквы и цифры
        self.pos_decoder = {"ADJF": "ADJ",
                            'ADJS': 'ADJ',
                            'COMP': 'ADJ',
                            'INFN': 'VERB',
                            'PRTF': 'VERB'}

    def norm_word(self, word: str) -> str:
        w = str(re.sub(' +', ' ', self.regexp.sub(' ', word.lower()))).strip()
        if len(w) > 1 and w[0].isalpha():
            parsed = self.morph.parse(w)[0]
            tag = str(parsed.tag.POS)
            if tag not in ['NPRO', 'PRED', 'PREP', 'CONJ', 'PRCL', 'INTJ']:
                tag = tag if tag not in self.pos_decoder else self.pos_decoder[tag]
                return parsed.normal_form.replace('ё', 'e')
        return ''
    
    def norm_sentence(self, sentence: str):
        tokens = list(filter(lambda x: len(x), [self.norm_word(word) for word in self.tokenizer.tokenize(sentence.lower())]))
        # bigrams = ['_'.join(w) for w in  ngrams(tokens,n=2)]
        return tokens
    
        

In [7]:
prep = Preparator()
texts = [prep.norm_sentence(i) for i in tqdm(row_data['title'])]

  0%|          | 0/119228 [00:00<?, ?it/s]

In [8]:
row_data.dtypes

link         object
text         object
title        object
source       object
timestamp     int64
refers_to    object
dtype: object

In [9]:
list(row_data.timestamp)[0]

1665171180000

In [10]:
min(row_data.timestamp), max(row_data.timestamp)

(-6795364578871, 1665181237000)

In [23]:
row_data['normed_text'] = texts
row_data['slice'] = [datetime.datetime.utcfromtimestamp(ts//1000).year for ts in row_data['timestamp']]
train, test = train_test_split(row_data, test_size=0.7, random_state=42, shuffle = True)

In [24]:
row_data.slice.value_counts()

2022    32376
2018    12735
2021    10610
2019    10352
2020     9473
2017     9171
2016     7527
2015     5419
2008     4302
2009     3412
2011     3378
2010     3134
2014     2570
2012     2188
2013     2041
2007      536
1754        4
Name: slice, dtype: int64

In [25]:
list(test.timestamp)[0]

1296739500000

In [26]:
dictionary = gensim.corpora.Dictionary(train.normed_text)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in train.normed_text]
time_slice = train.slice
num_topics = 20

In [27]:
dictionary.num_pos, dictionary.num_docs, dictionary.num_nnz

(310298, 35768, 308721)

In [ ]:
%%time
ldaseq = LdaSeqModel(corpus=doc_term_matrix, id2word=dictionary, time_slice=time_slice, num_topics=num_topics)
with open('/home/jovyan/work/ldaseq2.pkl', 'wb') as f:
     pickle.dump(ldaseq, f)